In [1]:
from modules.lstm_encoder import LSTMEncoder
from modules.lstm_decoder import LSTMDecoder
from keras.regularizers import l2
import os
import numpy as np
import pandas as pd

Using TensorFlow backend.


## Encoding

In [2]:
encoder = LSTMEncoder(l2, './features/lstm/2_steps/', './convnet_weights/lstm/base/weights.19-1.13.hdf5')

In [3]:
features_dev = encoder.encode('dev')

26/26 [==============================] - 13s 508ms/step


In [4]:
features_dev.shape

(3313, 2, 2048)

## Classes

In [5]:
indices = np.load('./features/lstm/2_steps/indices_dev.npy')
true_classes = np.load('./features/res_net/classes_dev_make-model.npy')[indices]
true_classnames = np.load('./features/res_net/classnames_dev_make-model.npy')[indices]
true_filenames = np.load('./features/res_net/filenames_dev.npy')[indices]

In [6]:
true_classes

array([ 54,  89, 164, ..., 136,  14,   3])

In [7]:
true_classnames

array(['Chevrolet Silverado_1500', 'Ford F-450', 'Volvo 240', ...,
       'Mercedes-Benz 300-Class', 'Audi S6', 'Acura TL'],
      dtype='<U33')

In [9]:
true_filenames

array(['Chevrolet Silverado_1500 Regular_Cab 2012/02808.jpg',
       'Ford F-450 Crew_Cab 2012/05187.jpg',
       'Volvo 240 Sedan 1993/04113.jpg', ...,
       'Mercedes-Benz 300-Class Convertible 1993/07798.jpg',
       'Audi S6 Sedan 2011/00602.jpg', 'Acura TL Sedan 2012/03169.jpg'],
      dtype='<U58')

## Predictions

In [10]:
def map_predictions_to_classnames(predictions, true_classes, classnames):
    classes, indices = np.unique(true_classes, return_index=True)
    
    class_to_index = {}
    for cl, index in zip(classes, indices):
        class_to_index[cl] = index
    
    predicted_classnames = list(map(lambda cl: classnames[class_to_index[cl]], predictions))
    return np.array(predicted_classnames)

In [11]:
decoder = LSTMDecoder(None, None, features_dev, true_classes, weights_path='./convnet_weights/lstm/decoder_lstm/weights.36-1.27.hdf5')

In [12]:
predictions = decoder.predict('dev')

3313/3313 [==============================] - 6s 2ms/step


In [13]:
predicted_classnames = map_predictions_to_classnames(predictions, true_classes, true_classnames)

## Report

In [14]:
classes, indices = np.unique(true_classes, return_index=True)
classnames = true_classnames[indices]
directories = list(map(lambda f: f.split('/')[0],true_filenames[indices]))

In [44]:
classes_report = pd.DataFrame({
    'name': classnames,
    'directory': directories,
}, index = classes)

In [16]:
pred_classnames = list(map(lambda cl: classes_report['name'][cl], predictions))

In [48]:
samples_report = pd.DataFrame({
    'filename': true_filenames,
    'prediction': predictions,
    'real_class': true_classes,
    'predicted_name': pred_classnames,
    'real_name': true_classnames,
    'predicted_correctly': predictions == true_classes,
})

In [30]:
classes_count = samples_report['real_class'].value_counts()
classes_report['count'] = classes_count.sort_index()

In [35]:
correct_count = samples_report[samples_report['predicted_correctly'] == True]['real_class'].value_counts()
classes_report['correct_predictions_count'] = correct_count.sort_index()

In [39]:
classes_report['correct_predictions_percent'] = classes_report['correct_predictions_count'] / classes_report['count']

In [43]:
classes_report

,class,name,directory,count,correct_predictions_count,correct_predictions_percent
0,0,AM Hummer,AM Hummer SUV 2000,18,17,0.944444
1,1,Acura Integra_Type_R,Acura Integra_Type_R Coupe 2001,18,12,0.666667
2,2,Acura RL,Acura RL Sedan 2012,13,4,0.307692
3,3,Acura TL,Acura TL Type-S 2008,35,21,0.600000
4,4,Acura TSX,Acura TSX Sedan 2012,17,7,0.411765
5,5,Acura ZDX,Acura ZDX Hatchback 2012,16,8,0.500000
6,6,Aston_Martin V8_Vantage,Aston_Martin V8_Vantage Convertible 2012,35,21,0.600000
7,7,Aston_Martin Virage,Aston_Martin Virage Coupe 2012,30,19,0.633333
8,8,Audi 100,Audi 100 Sedan 1994,34,25,0.735294
9,9,Audi A5,Audi A5 Coupe 2012,17,7,0.411765
